URL file model h5 : https://drive.google.com/file/d/1JUsB8qG6c_XMzQqIV97WF40HWDBd5Yue/view?usp=drive_link

## **Preprocesing**

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('(main) book_data.csv')

In [3]:
df.head(2)

,books_id,title,url_playbook,url_image,synopsis,idr,isbn,author,avg_rating,tags1,tags2,tags3
0,1,Harry Potter and the Half-Blood Prince by J.K....,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439785960,"J.K. Rowling, Mary GrandPrÃ©","4,54",fantasy,fiction,young-adult
1,2,Harry Potter and the Order of the Phoenix by J...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439358078,"J.K. Rowling, Mary GrandPrÃ©","4,46",children,children-s,fantasy


In [4]:
df.shape

(6205, 12)

**Cleaning Data**

In [5]:
df.isnull().sum()

books_id          0
title             0
url_playbook      0
url_image         0
synopsis         75
idr               0
isbn              0
author          339
avg_rating        0
tags1             0
tags2             0
tags3             0
dtype: int64

In [6]:
df= df.dropna()

In [7]:
df.isnull().sum()

books_id        0
title           0
url_playbook    0
url_image       0
synopsis        0
idr             0
isbn            0
author          0
avg_rating      0
tags1           0
tags2           0
tags3           0
dtype: int64

In [8]:
df.duplicated().sum()

0

**Penggabungan Columns Genre dan Tags**

In [9]:
df["genre"] = df["tags1"].astype(str) + ", " + df["tags2"].astype(str) + ", " + df["tags3"].astype(str)
df

,books_id,title,url_playbook,url_image,synopsis,idr,isbn,author,avg_rating,tags1,tags2,tags3,genre
0,1,Harry Potter and the Half-Blood Prince by J.K....,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439785960,"J.K. Rowling, Mary GrandPrÃ©","4,54",fantasy,fiction,young-adult,"fantasy, fiction, young-adult"
1,2,Harry Potter and the Order of the Phoenix by J...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439358078,"J.K. Rowling, Mary GrandPrÃ©","4,46",children,children-s,fantasy,"children, children-s, fantasy"
2,3,"""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",https://play.google.com/store/books/details/Jo...,https://books.google.com/books/publisher/conte...,This instrumental series contains selected the...,"$97,297.00",439554934,"J.K. Rowling, Mary GrandPrÃ©","4,44",fantasy,fiction,young-adult,"fantasy, fiction, young-adult"
3,5,Harry Potter and the Prisoner of Azkaban (Nove...,https://play.google.com/store/books/details/Ke...,https://books.google.com/books/publisher/conte...,Harry Potter and the Prisoner of Azkaban is ab...,"$145,126.00",439655480,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult"
4,6,Harry Potter and the Goblet of Fire by J.K. Ro...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439139600,"J.K. Rowling, Mary GrandPrÃ©","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6197,30008702,The 16 Personality Types in a Nutshell,https://play.google.com/store/books/details/Ja...,https://books.google.com/books/publisher/conte...,<b>Meet the self-sufficient and brilliant stra...,$0.00,385542070,Ian McEwan,"3,71",thriller,book-club,contemporary,"thriller, book-club, contemporary"
6198,30253864,Talking as Fast as I Can: From Gilmore Girls t...,https://play.google.com/store/books/details/La...,https://books.google.com/books/publisher/conte...,<b><i>NEW YORK TIMES</i> BESTSELLER â€¢ In thi...,"$148,851.00",425285170,Lauren Graham,"4,04",fiction,memoir,non-fiction,"fiction, memoir, non-fiction"
6200,30555488,The Underground Railroad: Winner of the Pulitz...,https://play.google.com/store/books/details/Co...,https://books.google.com/books/publisher/conte...,<b>WINNER OF THE PULITZER PRIZE FOR FICTION 20...,"$123,598.00",385542364,Colson Whitehead,"4,04",book-club,fiction,historical-fiction,"book-club, fiction, historical-fiction"
6203,31845516,Love Warrior: A Memoir,https://play.google.com/store/books/details/Gl...,https://books.google.com/books/publisher/conte...,<b>#1 <i>New York Times</i> Bestseller | </b><...,"$178,651.00",1250128544,Glennon Doyle Melton,"4,1",memoir,non-fiction,nonfiction,"memoir, non-fiction, nonfiction"


In [10]:
df['tag'] = df['title'] + df['synopsis'] + df['author'] + df['genre']
df

,books_id,title,url_playbook,url_image,synopsis,idr,isbn,author,avg_rating,tags1,tags2,tags3,genre,tag
0,1,Harry Potter and the Half-Blood Prince by J.K....,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439785960,"J.K. Rowling, Mary GrandPrÃ©","4,54",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Half-Blood Prince by J.K....
1,2,Harry Potter and the Order of the Phoenix by J...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439358078,"J.K. Rowling, Mary GrandPrÃ©","4,46",children,children-s,fantasy,"children, children-s, fantasy",Harry Potter and the Order of the Phoenix by J...
2,3,"""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",https://play.google.com/store/books/details/Jo...,https://books.google.com/books/publisher/conte...,This instrumental series contains selected the...,"$97,297.00",439554934,"J.K. Rowling, Mary GrandPrÃ©","4,44",fantasy,fiction,young-adult,"fantasy, fiction, young-adult","""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ..."
3,5,Harry Potter and the Prisoner of Azkaban (Nove...,https://play.google.com/store/books/details/Ke...,https://books.google.com/books/publisher/conte...,Harry Potter and the Prisoner of Azkaban is ab...,"$145,126.00",439655480,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Prisoner of Azkaban (Nove...
4,6,Harry Potter and the Goblet of Fire by J.K. Ro...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439139600,"J.K. Rowling, Mary GrandPrÃ©","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Goblet of Fire by J.K. Ro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6197,30008702,The 16 Personality Types in a Nutshell,https://play.google.com/store/books/details/Ja...,https://books.google.com/books/publisher/conte...,<b>Meet the self-sufficient and brilliant stra...,$0.00,385542070,Ian McEwan,"3,71",thriller,book-club,contemporary,"thriller, book-club, contemporary",The 16 Personality Types in a Nutshell<b>Meet ...
6198,30253864,Talking as Fast as I Can: From Gilmore Girls t...,https://play.google.com/store/books/details/La...,https://books.google.com/books/publisher/conte...,<b><i>NEW YORK TIMES</i> BESTSELLER â€¢ In thi...,"$148,851.00",425285170,Lauren Graham,"4,04",fiction,memoir,non-fiction,"fiction, memoir, non-fiction",Talking as Fast as I Can: From Gilmore Girls t...
6200,30555488,The Underground Railroad: Winner of the Pulitz...,https://play.google.com/store/books/details/Co...,https://books.google.com/books/publisher/conte...,<b>WINNER OF THE PULITZER PRIZE FOR FICTION 20...,"$123,598.00",385542364,Colson Whitehead,"4,04",book-club,fiction,historical-fiction,"book-club, fiction, historical-fiction",The Underground Railroad: Winner of the Pulitz...
6203,31845516,Love Warrior: A Memoir,https://play.google.com/store/books/details/Gl...,https://books.google.com/books/publisher/conte...,<b>#1 <i>New York Times</i> Bestseller | </b><...,"$178,651.00",1250128544,Glennon Doyle Melton,"4,1",memoir,non-fiction,nonfiction,"memoir, non-fiction, nonfiction",Love Warrior: A Memoir<b>#1 <i>New York Times<...


**Mengubah Column Tags menjadi Kata Dasar**

In [12]:
import re
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
# Function for removing NonAscii characters
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)
# Function for converting into lower case
def make_lower_case(text):
    return text.lower()
# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text
# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text
#Function for removing the html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
# Applying all the functions in description and storing as a cleaned_desc
df['tags'] = df['tag'].apply(_removeNonAscii)
df['tags'] = df.tags.apply(func = make_lower_case)
df['tags'] = df.tags.apply(func = remove_stop_words)
df['tags'] = df.tags.apply(func=remove_punctuation)
df['tags'] = df.tags.apply(func=remove_html)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
df

,books_id,title,url_playbook,url_image,synopsis,idr,isbn,author,avg_rating,tags1,tags2,tags3,genre,tag,tags
0,1,Harry Potter and the Half-Blood Prince by J.K....,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439785960,"J.K. Rowling, Mary GrandPrÃ©","4,54",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Half-Blood Prince by J.K....,harry potter half blood prince j k rowling boo...
1,2,Harry Potter and the Order of the Phoenix by J...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439358078,"J.K. Rowling, Mary GrandPrÃ©","4,46",children,children-s,fantasy,"children, children-s, fantasy",Harry Potter and the Order of the Phoenix by J...,harry potter order phoenix j k rowling book an...
2,3,"""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",https://play.google.com/store/books/details/Jo...,https://books.google.com/books/publisher/conte...,This instrumental series contains selected the...,"$97,297.00",439554934,"J.K. Rowling, Mary GrandPrÃ©","4,44",fantasy,fiction,young-adult,"fantasy, fiction, young-adult","""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",harry potter sorcerer s stone selected themes ...
3,5,Harry Potter and the Prisoner of Azkaban (Nove...,https://play.google.com/store/books/details/Ke...,https://books.google.com/books/publisher/conte...,Harry Potter and the Prisoner of Azkaban is ab...,"$145,126.00",439655480,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Prisoner of Azkaban (Nove...,harry potter prisoner azkaban novel study gr 4...
4,6,Harry Potter and the Goblet of Fire by J.K. Ro...,https://play.google.com/store/books/details/Br...,https://books.google.com/books/publisher/conte...,<b>Unlock the more straightforward side of <i>...,"$81,056.00",439139600,"J.K. Rowling, Mary GrandPrÃ©","4,53",fantasy,fiction,young-adult,"fantasy, fiction, young-adult",Harry Potter and the Goblet of Fire by J.K. Ro...,harry potter goblet fire j k rowling book anal...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6197,30008702,The 16 Personality Types in a Nutshell,https://play.google.com/store/books/details/Ja...,https://books.google.com/books/publisher/conte...,<b>Meet the self-sufficient and brilliant stra...,$0.00,385542070,Ian McEwan,"3,71",thriller,book-club,contemporary,"thriller, book-club, contemporary",The 16 Personality Types in a Nutshell<b>Meet ...,16 personality types nutshell b meet self suff...
6198,30253864,Talking as Fast as I Can: From Gilmore Girls t...,https://play.google.com/store/books/details/La...,https://books.google.com/books/publisher/conte...,<b><i>NEW YORK TIMES</i> BESTSELLER â€¢ In thi...,"$148,851.00",425285170,Lauren Graham,"4,04",fiction,memoir,non-fiction,"fiction, memoir, non-fiction",Talking as Fast as I Can: From Gilmore Girls t...,talking fast can gilmore girls gilmore girls a...
6200,30555488,The Underground Railroad: Winner of the Pulitz...,https://play.google.com/store/books/details/Co...,https://books.google.com/books/publisher/conte...,<b>WINNER OF THE PULITZER PRIZE FOR FICTION 20...,"$123,598.00",385542364,Colson Whitehead,"4,04",book-club,fiction,historical-fiction,"book-club, fiction, historical-fiction",The Underground Railroad: Winner of the Pulitz...,underground railroad winner pulitzer prize fic...
6203,31845516,Love Warrior: A Memoir,https://play.google.com/store/books/details/Gl...,https://books.google.com/books/publisher/conte...,<b>#1 <i>New York Times</i> Bestseller | </b><...,"$178,651.00",1250128544,Glennon Doyle Melton,"4,1",memoir,non-fiction,nonfiction,"memoir, non-fiction, nonfiction",Love Warrior: A Memoir<b>#1 <i>New York Times<...,love warrior memoir b 1 i new york times i bes...


**Inisialisasi Tabel Baru yang akan Digunakan Untuk Rekomendasi**

In [14]:
df_fix = df[['books_id', 'title', 'tags']]

In [15]:
df_fix.head(5)

,books_id,title,tags
0,1,Harry Potter and the Half-Blood Prince by J.K....,harry potter half blood prince j k rowling boo...
1,2,Harry Potter and the Order of the Phoenix by J...,harry potter order phoenix j k rowling book an...
2,3,"""Harry Potter and the Sorcerer's Stoneâ„¢"" -- ...",harry potter sorcerer s stone selected themes ...
3,5,Harry Potter and the Prisoner of Azkaban (Nove...,harry potter prisoner azkaban novel study gr 4...
4,6,Harry Potter and the Goblet of Fire by J.K. Ro...,harry potter goblet fire j k rowling book anal...


**Stemming**

Mengubah data menjadi Kata Dasar

In [16]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [17]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

In [19]:
df_fix['tags'].apply(stem)

0       harri potter half blood princ j k rowl book an...
1       harri potter order phoenix j k rowl book analy...
2       harri potter sorcer s stone select theme motio...
3       harri potter prison azkaban novel studi gr 4 8...
4       harri potter goblet fire j k rowl book analysi...
                              ...                        
6197    16 person type nutshel b meet self suffici bri...
6198    talk fast can gilmor girl gilmor girl and ever...
6200    underground railroad winner pulitz prize ficti...
6203    love warrior memoir b 1 i new york time i best...
6204    hate u give printz honor winner8 star review g...
Name: tags, Length: 5797, dtype: object

In [20]:
df_fix['tags'] = df_fix['tags'].apply(stem)

<ipython-input-20-f3c16866a5ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fix['tags'] = df_fix['tags'].apply(stem)


## **Count Vectorizer**

**Mengubah Columns Tags menjadi bentuk Vector**

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv  = CountVectorizer(max_features=6000 , stop_words= 'english')


**Transform**

Metode fit_transform digunakan untuk mempelajari kosakata dari teks dalam kolom 'tags' dan mengubahnya menjadi vektor representasi. Proses ini melibatkan tokenisasi teks, menghapus kata penghubung (stop words), dan menghitung frekuensi kemunculan kata-kata dalam setiap dokumen (teks).

Kemudian hasil dari fit_transform dikonversi menjadi bentuk array numpy. Setiap baris dalam array ini akan mewakili vektor representasi untuk setiap dokumen (teks) dalam DataFrame.

In [22]:
cv.fit_transform(df_fix['tags']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
vectors = cv.fit_transform(df_fix['tags']).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
cv.get_feature_names_out()

array(['000', '10', '100', ..., 'zombi', 'zone', 'zoo'], dtype=object)

In [25]:
vectors.shape

(5797, 6000)

## **Proses Similarity**

In [26]:
import tensorflow as tf
vectors = vectors.astype(np.float32)  # Mengonversi ke tipe data float32

vectors_norm = tf.linalg.norm(vectors, axis=1, keepdims=True)
similarity = tf.matmul(vectors, vectors, transpose_b=True) / (vectors_norm * vectors_norm)

In [27]:
similarity = similarity.numpy()

In [28]:
similarity

array([[1.        , 0.89193547, 0.29354838, ..., 0.816129  , 0.43709677,
        0.08387097],
       [0.8366111 , 0.9999999 , 0.28744325, ..., 0.8108925 , 0.4447806 ,
        0.07866868],
       [1.2999998 , 1.3571427 , 0.9999999 , ..., 1.5857141 , 0.8285713 ,
        0.08571427],
       ...,
       [0.4223706 , 0.44741234, 0.18530884, ..., 1.        , 0.3806344 ,
        0.09599333],
       [0.5634095 , 0.61122656, 0.24116422, ..., 0.94802487, 0.99999994,
        0.13305612],
       [0.23636365, 0.23636365, 0.05454546, ..., 0.5227273 , 0.2909091 ,
        1.0000001 ]], dtype=float32)

In [29]:
similarity.shape

(5797, 5797)

In [30]:
similarity[0]

array([1.        , 0.89193547, 0.29354838, ..., 0.816129  , 0.43709677,
       0.08387097], dtype=float32)

In [31]:
similarity[1]

array([0.8366111 , 0.9999999 , 0.28744325, ..., 0.8108925 , 0.4447806 ,
       0.07866868], dtype=float32)

## **Fungsi Rekomendasi**

In [32]:
# def recommend(book):
#     index = np.where(df_fix['books_id'] == book)[0][0]
#     similar_books = sorted(enumerate(similarity[index]) , key=lambda x:x[1]   , reverse=True)[1:11]

#     for i in similar_books:
#         print (df_fix['books_id'][i[0]])
def recommend(book1, book2, book3):
    books = [book1, book2, book3]
    similarity_combined = np.zeros(len(df_fix))  # Inisialisasi similarity_combined dengan array nol

    for book in books:
        index = np.where(df_fix['books_id'] == book)[0]
        if len(index) > 0:
            index = index[0]
            similarity_combined += similarity[index]  # Menambahkan similarity dari setiap buku

    similar_books = sorted(enumerate(similarity_combined), key=lambda x: x[1], reverse=True)[1:11]
    for book in similar_books:
        print(df_fix['books_id'][book[0]])

Uji Coba

In [33]:
recommend(1, 6670, 7926)

11574
7257982
49465
49251
8765461
61329
11275
119
1914973
7039218


## **Model h5**

In [34]:
import h5py
from sklearn.preprocessing import StandardScaler

# Hapus kolom dengan nilai string dari DataFrame
df_numeric = df_fix.select_dtypes(include=[np.number])

# Skalakan DataFrame
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

# Simpan DataFrame dan similarity matrix ke dalam file H5
with h5py.File('model.h5', 'w') as hf:
    hf.create_dataset('df_scaled', data=df_scaled)
    hf.create_dataset('similarity', data=similarity)